In [8]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np

In [545]:
def gt_fn(x):
    #print x
    x = x.reshape(-1,4)
    return (np.max(x[:,[0,1]], axis=1) > ((x[:,2]) + x[:,3]/2.0))*1.0

In [546]:
gt_fn(np.random.rand(2,1,4))

array([ 1.,  0.])

In [549]:
X_train = np.random.rand(100000,4)*10
y_train = gt_fn(X_train)

X_test = np.random.rand(10000,4)*10
y_test = gt_fn(X_test)

In [548]:
import sklearn
def sample(X_batch, y_batch, size, num_classes):
    
    X_batch = np.asarray(X_batch)
    y_batch = np.asarray(y_batch)
    
    x_data = []
    y_data = []
    
    for i in range(num_classes):
        x_data.append(sklearn.utils.resample(X_batch[np.where(y_batch == i)[0]], n_samples=size))
        y_data.append(np.zeros((size, 1))+i)


    X_batch = np.concatenate(x_data)
    y_batch = np.concatenate(y_data)

    todo = range(len(y_batch))
    np.random.shuffle(todo)

    X_batch = X_batch[todo]
    y_batch = y_batch[todo]
    
    return X_batch, y_batch

#sample(X_train, y_train, 10,2)

In [114]:
edges_np = np.array([(0,3),(3,1),(1,2),(2,3)])
edges_np = np.concatenate([edges_np,edges_np[:,[1,0]]])

In [115]:
edges_np

array([[0, 3],
       [3, 1],
       [1, 2],
       [2, 3],
       [3, 0],
       [1, 3],
       [2, 1],
       [3, 2]])

In [13]:
edge_selector = np.array([np.where(edges_np[:,0] == i)[0] for i in range(4)])

In [14]:
edge_selector

array([array([0]), array([2, 5]), array([3, 6]), array([1, 4, 7])], dtype=object)

In [59]:
edges = torch.LongTensor(edges_np)

In [60]:
edges


    0     3
    3     1
    1     2
    2     3
    3     0
    1     3
    2     1
    3     2
[torch.LongTensor of size 8x2]

In [71]:
weights = Variable(torch.rand(edges.shape), requires_grad=True)

In [72]:
weights

Variable containing:
 0.5684  0.1352
 0.0188  0.3241
 0.6176  0.1497
 0.6121  0.2223
 0.6169  0.3865
 0.9437  0.9026
 0.6818  0.4499
 0.3595  0.6131
[torch.FloatTensor of size 8x2]

In [19]:
# tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(2,-1, 2)
# tocompute

In [491]:
class CGN2(nn.Module):
    def __init__(self,input_dim, edges, out_dim=1, on_cuda=True):
        super(CGN2, self).__init__()

        self.my_layers = []
        self.out_dim = out_dim
        self.on_cuda = on_cuda
        self.edges = edges
        #dims = [input_dim] + channels
        
        print "Constructing the network..."   
        
        
        self.weights1 = nn.Parameter(torch.rand(edges.shape), requires_grad=True)
        print self.weights1.size()

        self.last_layer = nn.Linear(input_dim, out_dim)
        self.my_layers = nn.ModuleList([self.last_layer])

        print "Done!"

        
    #batch_size = 2
    #print x
    def GraphConv(self, x, edges, batch_size, weights):
        
        x = x.clone()
        
        tocompute = torch.index_select(x, 1, Variable(edges.view(-1))).view(batch_size, -1, 2)
        #print tocompute
        conv = tocompute*weights
        #print conv
        for i, edges_to_select in enumerate(edge_selector):
            #print "x", conv
            #print "e", edges_to_select
            selected_edges = torch.index_select(conv, 1, Variable(torch.LongTensor(edges_to_select)))
            #print "m", selected_edges
            selected_edges = selected_edges.view(-1,edges_to_select.shape[0]*2)
            #print "m", selected_edges
            pooled_edges = torch.max(selected_edges,1)[0]
            #print "mmo",pooled_edges
            x[:,i] = pooled_edges
            #print "xx",x[:,i]
        return x

        
        
    def forward(self, x):
        nb_examples, nb_nodes = x.size()
        
        self.GraphConv(x,self.edges, nb_examples, self.weights1)

        x = self.last_layer(x.view(nb_examples, -1))
        x = F.sigmoid(x)

        return x

In [550]:
import models
reload(models);

In [551]:
#gc = CGN2(4,edges)
gc = models.MLP(4,[10],2)
gc

Constructing the network...
Done!


MLP (
  (my_layers): ModuleList (
    (0): Linear (4 -> 10)
  )
  (last_layer): Linear (10 -> 2)
)

In [586]:
list(gc.parameters())

[Parameter containing:
  0.3155 -0.4150 -0.3406 -0.4055
  0.1289  0.1336 -0.1016  0.4493
 -0.4373  0.4367 -0.0760 -0.0526
 -0.2413  0.2277  0.3490  0.2422
 -0.4667 -0.1930  0.4590 -0.3802
 -0.1446 -0.0561 -0.1433 -0.1082
 -0.4837  0.0318 -0.3148  0.3454
 -0.0987  0.0363  0.4293  0.1803
 -0.4004  0.1092  0.4453 -0.4015
  0.3695 -0.4080 -0.0458 -0.4440
 [torch.FloatTensor of size 10x4], Parameter containing:
 -0.1733
 -0.4135
 -0.2673
 -0.2628
  0.1145
  0.3395
 -0.4669
  0.0224
 -0.4844
  0.0131
 [torch.FloatTensor of size 10], Parameter containing:
 -0.0450  0.0948 -0.2732  0.0282 -0.1569 -0.2957 -0.1764  0.0507 -0.1561  0.1715
 -0.2333 -0.0783 -0.3086 -0.0057 -0.2432  0.3046  0.0749 -0.1615  0.2999 -0.0795
 [torch.FloatTensor of size 2x10], Parameter containing:
  0.3101
 -0.2874
 [torch.FloatTensor of size 2]]

In [578]:
optimizer = optim.SGD(list(gc.parameters()), lr = 0.001, momentum=0.9)

In [583]:
x = Variable(torch.FloatTensor(X_train[:8])).view(-1,1,4)
x

Variable containing:
(0 ,.,.) = 
  1.5376  4.6742  4.4718  7.1636

(1 ,.,.) = 
  3.6882  6.1942  7.1988  6.8813

(2 ,.,.) = 
  1.5468  6.3984  2.5569  6.5676

(3 ,.,.) = 
  4.7741  7.6312  5.4124  0.7692

(4 ,.,.) = 
  2.3023  8.2242  7.6240  0.6494

(5 ,.,.) = 
  6.1428  9.9681  7.9972  1.8933

(6 ,.,.) = 
  4.4662  8.0372  8.6390  5.4916

(7 ,.,.) = 
  5.0784  3.2834  0.3930  1.1739
[torch.FloatTensor of size 8x1x4]

In [584]:
output = gc(x)
output

Variable containing:
 0.8122  0.1878
 0.8366  0.1634
 0.7971  0.2029
 0.7441  0.2559
 0.7160  0.2840
 0.7339  0.2661
 0.7758  0.2242
 0.8336  0.1664
[torch.FloatTensor of size 8x2]

In [585]:
gt = Variable(torch.FloatTensor(gt_fn(x.data.numpy()))).view(-1,1)
loss = torch.abs(gt-output)
print loss
print loss.mean()

Variable containing:
 0.8122  0.1878
 0.8366  0.1634
 0.2029  0.7971
 0.2559  0.7441
 0.2840  0.7160
 0.2661  0.7339
 0.7758  0.2242
 0.1664  0.8336
[torch.FloatTensor of size 8x2]

Variable containing:
 0.5000
[torch.FloatTensor of size 1]



In [582]:
loss.mean().backward()
optimizer.step()

In [261]:
X_batch, y_batch = sample(X_train, y_train, 1,2)

In [262]:
X_batch.shape

(2, 4)

In [263]:
x = Variable(torch.FloatTensor(X_batch))
output = gc(x)
#gt = Variable(torch.FloatTensor(gt_fn(x.data.numpy()))).view(batch_size,-1)
#loss = torch.abs(gt-output)
#loss

In [247]:
y_batch

array([[ 0.],
       [ 1.],
       [ 1.],
       [ 0.]])

In [57]:
torch.manual_seed(0)
x = Variable(torch.rand(batch_size,4)*10)
output = torch.nn.Sigmoid()(linearLayer(x)).view(-1)
loss = torch.abs(Variable(gt_fn(x.data))-output)
print  loss

Variable containing:
 0.1381
 0.1273
[torch.FloatTensor of size 2]



In [58]:
loss.mean().backward()
optimizer.step()